## Part 1: Preprocessing - Aditya

In [26]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [5]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [20]:
# Create a list of at least 10 column names to use as X data
X_columns = list(attrition_df.drop(['Attrition', 'Department'], axis=1).columns[:10])


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Age                         int64
BusinessTravel             object
DistanceFromHome            int64
Education                   int64
EducationField             object
EnvironmentSatisfaction     int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
JobRole                    object
dtype: object

In [83]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)


In [84]:
print('BusinessTravel', list(X_train['BusinessTravel'].unique()))
print('EducationField', list(X_train['EducationField'].unique()))
print('JobRole', list(X_train['JobRole'].unique()))

BusinessTravel ['Travel_Frequently', 'Travel_Rarely', 'Non-Travel']
EducationField ['Medical', 'Human Resources', 'Life Sciences', 'Marketing', 'Technical Degree', 'Other']
JobRole ['Sales Executive', 'Laboratory Technician', 'Human Resources', 'Healthcare Representative', 'Research Scientist', 'Manager', 'Manufacturing Director', 'Research Director', 'Sales Representative']


In [85]:
# Encoding categorical columns in X dataset
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoder.set_output(transform="pandas")
# Fit_transform the eyes column
encoder.fit(X_train[['BusinessTravel', 'EducationField', 'JobRole']])
X_train_encoded = encoder.transform(X_train[['BusinessTravel', 'EducationField', 'JobRole']])
X_test_encoded = encoder.transform(X_test[['BusinessTravel', 'EducationField', 'JobRole']])


In [86]:
# Updating X datasets with encoded columns
X_train_encoded = pd.concat([X_train.drop(['BusinessTravel', 'EducationField', 'JobRole'], axis=1), X_train_encoded], axis=1)
X_test_encoded = pd.concat([X_test.drop(['BusinessTravel', 'EducationField', 'JobRole'], axis=1), X_test_encoded], axis=1)



In [87]:
# Rechecking X dataset datatypes
print(X_train_encoded.dtypes)
print(X_test_encoded.dtypes)


Age                                   int64
DistanceFromHome                      int64
Education                             int64
EnvironmentSatisfaction               int64
HourlyRate                            int64
JobInvolvement                        int64
JobLevel                              int64
BusinessTravel_Travel_Frequently    float64
BusinessTravel_Travel_Rarely        float64
EducationField_Life Sciences        float64
EducationField_Marketing            float64
EducationField_Medical              float64
EducationField_Other                float64
EducationField_Technical Degree     float64
JobRole_Human Resources             float64
JobRole_Laboratory Technician       float64
JobRole_Manager                     float64
JobRole_Manufacturing Director      float64
JobRole_Research Director           float64
JobRole_Research Scientist          float64
JobRole_Sales Executive             float64
JobRole_Sales Representative        float64
dtype: object
Age               

In [88]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train_encoded.iloc[:,:7])

# Scale the training and testing data
X_train_encoded.iloc[:,:7] = scaler.transform(X_train_encoded.iloc[:,:7])
X_test_encoded.iloc[:,:7] = scaler.transform(X_test_encoded.iloc[:,:7])


In [89]:
# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = encoder.transform(y_train[['Department']])
y_test_department = encoder.transform(y_test[['Department']])


In [90]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = encoder.transform(y_train[['Attrition']])
y_test_attrition = encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [101]:
# Find the number of columns in the X training data
input_shape = len(X_train_encoded.columns)

# Create the input layer
input_layer = layers.Input(shape=(input_shape,), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)


In [102]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_dense = layers.Dense(32, activation='relu', name='department_dense')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3,
                             activation='sigmoid',
                             name='department_output')(department_dense)


In [103]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(32, activation='relu', name='attrition_dense')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2,
                             activation='sigmoid',
                             name='attrition_output')(attrition_dense)

In [117]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 22)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │      1,472 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_dense    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_dense     │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_dense… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_dense[… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 18,213 (71.14 KB)

 Trainable params: 18,213 (71.14 KB)

 Non-trainable params: 0 (0.00 B)

In [119]:
# Train the model
model.fit(
    X_train_encoded,
    {'department_output': y_train_department, 'attrition_output': y_train_attrition},
    epochs=10,
    batch_size=32,
)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8839 - department_output_accuracy: 0.9863 - loss: 0.3536
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - attrition_output_accuracy: 0.8902 - department_output_accuracy: 0.9823 - loss: 0.3232
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - attrition_output_accuracy: 0.8887 - department_output_accuracy: 0.9891 - loss: 0.3099
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - attrition_output_accuracy: 0.8941 - department_output_accuracy: 0.9933 - loss: 0.2726
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - attrition_output_accuracy: 0.8978 - department_output_accuracy: 0.9874 - loss: 0.2850
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - attrition_output_accuracy: 0.9088 - department_output_accuracy: 0.9924 - loss: 0.2841
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 535us/step - attrition_output_accuracy: 0.8835 - department_output_accuracy: 0.9916 - loss: 0.2953
Epoch 8/10
35/35 ━━━━━

In [120]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_encoded, {'department_output': y_test_department, 'attrition_output': y_test_attrition})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 536us/step - attrition_output_accuracy: 0.7954 - department_output_accuracy: 0.9830 - loss: 0.6863


[0.6234400272369385, 0.79076087474823, 0.98097825050354]

In [122]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[1]}")
print(f"Attrition Accuracy: {test_results[2]}")

Department Accuracy: 0.79076087474823
Attrition Accuracy: 0.98097825050354


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Initially, accuracy is a good startin point but it may not be the most informative because there may be an imablance of classes. The model might simply predict the majority class most of the time and still achieve high accuracy, even if it performs poorly for less frequent classes. Multi-class accuracy may be more informative.

2. Department output (sigmoid): The department output has multiple categories. Sigmoid activation is a good choice for multi-class classification problems where the output represents probabilities between 0 and 1 for each class. Attrition output (sigmoid): Attrition is a binary classification problem (leave or stay). Sigmoid activation is again suitable here as it outputs a value between 0 and 1, which can be interpreted as the probability of an employee leaving.

3. Hyperparameter tuning could help. Experimenting with different hyperparameters like the number of neurons in hidden layers or batch size could possibly improve performance. More complex architectures may improve resultds. Deeper networks with additional hidden layers and more neurons may improve results. Oversampling or undersampling to address class distribution imbalance could help improve performance for less frequent classes.